In [4]:
#Let's try different potentials

def One_over_R_potential(G,A):
    p = 0
    x = []
    for a in Subsets(A,2):
        x.append(list(a))
    for i in IntegerRange(len(x)):
        p = p + 1.0/G.distance(x[i][0],x[i][1])
    return p

def One_over_sqrtR_potential(G,A):
    p = 0
    x = []
    for a in Subsets(A,2):
        x.append(list(a))
    for i in IntegerRange(len(x)):
        p = p + 1.0/(sqrt(G.distance(x[i][0],x[i][1])))
    return p

def Log_potential(G,A):
    p = 0
    x = []
    for a in Subsets(A,2):
        x.append(list(a))
    for i in IntegerRange(len(x)):
        p = p + log(1.0/G.distance(x[i][0],x[i][1]))
    return p

def Negative_R_potential(G,A):
    p = 0
    x = []
    for a in Subsets(A,2):
        x.append(list(a))
    for i in IntegerRange(len(x)):
        p = p - G.distance(x[i][0],x[i][1])
    return p

def One_over_R2_potential(G,A):
    p = 0
    x = []
    for a in Subsets(A,2):
        x.append(list(a))
    for i in IntegerRange(len(x)):
        p = p + 1.0/(G.distance(x[i][0],x[i][1])^2)
    return p

def SumOfDistances(G,A):
    s = 0
    x = [list(a) for a in Subsets(A,2)]
    for i in range(len(x)):
        s = s + G.distance(x[i][0],x[i][1])
    return s

def image(s,X):
    Y = []
    for x in X:
        Y.append(s(x))
    return set(Y)
    
def imagelist(g,v):
    w = []
    for X in v:
        for s in g:
            w.append(image(s,X))
    return w


def DistinctGlobalMinimizers(G,k,potential):
    eMin = G.order()^2
    mins = []
    distinctmins = []
    automorphicimages = []
    g = G.automorphism_group()
    for X in Subsets(G,k):
        E = potential(G,X)
        if E==eMin:
            mins.append(X)
        if E<eMin:
            eMin=E
            mins=[X]
    distinctmins.append(mins[0])
    for i in IntegerRange(len(mins)):
        if i > 0:
            if mins[i] not in imagelist(g,distinctmins):
                distinctmins.append(mins[i])
    return distinctmins

    
def Perturbations(G,A):
    perts = []
    perts.append(A)
    for a in A:
        for u in G.vertices():
            if G.has_edge((a,u)) and u not in A:
                perts.append(list(Set(A).difference(Set([a])).union(Set([u]))))
    return perts
    
def GradientStep(G,A):
    perts = Perturbations(G,A)
    energies = [PotentialEnergy(G,P) for P in perts]
    return perts[energies.index(min(energies))]

def GradientDescent(G,A,N):
    X = copy(A)
    for i in range(N):
        X = GradientStep(G,X)
    return X
 
def ShortestPaths(D,u,v):
    shortestpaths = []
    for p in D.all_paths(u,v):
        if len(p)-1 == D.distance(u,v):
            shortestpaths.append(p)
    return shortestpaths

def SpanOfAPath(D,A,p):
    return (Set(A).intersection(Set(p))).cardinality()-1    

def ShortestPathsLengthSpectrum(D,A,k):
    spectrum = []
    for y in Subsets(A,2):
        x = list(y)
        for p in ShortestPaths(D,x[0],x[1]):
            if SpanOfAPath(D,A,p) == k:
                spectrum.append(Set(p).cardinality()-1)
    return spectrum

def AllSetsOfAGivenSpectrum(G,m,k,spec):
    output = []
    for A in Subsets(G.vertices(),m):
        spectrum = ShortestPathsLengthSpectrum(G,A,k)
        spectrum.sort()
        if spectrum == spec:
            output.append(A)
    return output


def SmallestOutside(vec):
    output = []
    vec.sort()
    for i in range(len(vec)):
        if i % 2 == 0:
            output = [vec[len(vec)-1]] + output
        if i % 2 == 1:
            output = output + [vec[len(vec)-1]]
        vec.remove(vec[len(vec)-1])
    return output

def SumOfSpectrum(n,m,k):
    if k == 1:
        return n-1
    onespec = [floor((n-1)/(m-1)),ceil((n-1)/(m-1))]
    if onespec[0] == onespec[1]:
        return sum([floor((n-1)/(m-1)) for i in range(m-1)])
    if onespec[0] != onespec[1]:
        onespecmu = [(m-1)*onespec[1]-(n-1),(n-1)-(m-1)*onespec[0]]
        onemultispec = []
        for i in range(onespecmu[0]):
            onemultispec.append(onespec[0])
        for i in range(onespecmu[1]):
            onemultispec.append(onespec[1])
        vec = []
        onemultispec = SmallestOutside(onemultispec)
        vec.append(onemultispec)
        mylist = copy(onemultispec)
        for i in range(floor((m-2)/2)):
            mylist.remove(mylist[0])
            mylist.remove(mylist[len(mylist)-1])
            vec.append(copy(mylist))
        if k > floor(m/2):
            k = m - k
        s = 0
        for i in range(k):
            s = s + sum(vec[i])
        return s
    
def MultiSpectrum(n,m,k):
    output = []
    s = SumOfSpectrum(n,m,k)
    spec = [floor(s/(m-k)),ceil(s/(m-k))]
    if spec[0] == spec[1]:
        return [spec[0] for i in range(m-k)]
    if spec[0] != spec[1]:
        specmu = [(m-k)*spec[1]-s,s-(m-k)*spec[0]]
        for i in range(specmu[0]):
            output.append(spec[0])
        for i in range(specmu[1]):
            output.append(spec[1])
        return output

def ToroidalGraph(n,m):
    g=graphs.GridGraph((n,m))
    for i in range(n):
        g.add_edge((i,0),(i,m-1))
    for i in range(m):
        if n > 1:
            g.add_edge((0,i),(n-1,i))
    #g.relabel()
    return g

def WheelieGraph(n):
    G = graphs.CycleGraph(n)
    for i in range(n):
        G.add_edge(i,(i+floor(n/2))%n)
    return G

def Power(G,k):
    K = copy(G)
    for v in G.vertices():
        for u in G.vertices():
            if u != v and G.distance(u,v) <= k:
                K.add_edge(u,v)
    return K

def WriteImages(n):
    Path = graphs.PathGraph(n)
    PathSquared = Power(Path,2)
    Cycle = graphs.CycleGraph(n)
    CycleSquared = Power(Cycle,2)
    Wheel = WheelieGraph(n)
    
    count = 0
    
    for m in range(n):
        MPath = DistinctGlobalMinimizers(Path,m+1,One_over_R_potential)
        MPathSquared = DistinctGlobalMinimizers(PathSquared,m+1,One_over_R_potential)
        MCycle = DistinctGlobalMinimizers(Cycle,m+1,One_over_R_potential)
        MCycleSquared = DistinctGlobalMinimizers(CycleSquared,m+1,One_over_R_potential)
        MWheel = DistinctGlobalMinimizers(Wheel,m+1,One_over_R_potential)
    
        MPath_R = DistinctGlobalMinimizers(Path,m+1,Negative_R_potential)
        MPathSquared_R = DistinctGlobalMinimizers(PathSquared,m+1,Negative_R_potential)
        MCycle_R = DistinctGlobalMinimizers(Cycle,m+1,Negative_R_potential)
        MCycleSquared_R = DistinctGlobalMinimizers(CycleSquared,m+1,Negative_R_potential)
        MWheel_R = DistinctGlobalMinimizers(Wheel,m+1,Negative_R_potential)
    
        my_graphs = [Cycle, CycleSquared, Wheel, Path, PathSquared]
        my_sets = [MCycle, MCycleSquared, MWheel, MPath, MPathSquared]
        my_sets_R = [MCycle_R, MCycleSquared_R, MWheel_R, MPath_R, MPathSquared_R]
        my_names = ['cycle', 'cyclesquared', 'wheel', 'path', 'pathsquared']

    
        #Create a folder
        import os, sys
        newpath = ((r'%s_pictures') % (n)) 
        if not os.path.exists(newpath): os.makedirs(newpath)

            
        
#        for i in range(len(my_graphs)):
#            if i == 0:
#                (my_graphs[i].plot(vertex_colors={'red':my_sets[i][j]}, figsize=3, axes=False) + text(str(n)+'_'+str(m+1)+'_'+str(my_names[i])+'_usual_energy_'+str(j),(0,-1.5))).save(str(n)+'_pictures/'+str(n)+'_'+str(m+1)+'_'+str(my_names[i])+'_usual_energy_'+str(j)+'.png')
#            if i > 0:
#                for j in range(len(my_sets[i])):
#                    (my_graphs[i].plot(vertex_colors={'blue':my_sets[i][j]}, figsize=3, axes=False) + text(str(n)+'_'+str(m+1)+'_'+str(my_names[i])+'_usual_energy_'+str(j),(0,-1.5))).save(str(n)+'_pictures/'+str(n)+'_'+str(m+1)+'_'+str(my_names[i])+'_usual_energy_'+str(j)+'.png')
#            for j in range(len(my_sets_R[i])):
#                (my_graphs[i].plot(vertex_colors={'green':my_sets_R[i][j]}, figsize=3, axes=False) + text(str(n)+'_'+str(m+1)+'_'+str(my_names[i])+'_sum_energy_'+str(j),(0,-1.5))).save(str(n)+'_pictures/'+str(n)+'_'+str(m+1)+'_'+str(my_names[i])+'_sum_energy_'+str(j)+'.png')
        
        #save images for HTML
        
        for i in range(len(my_graphs)):
            if i == 0:
                for j in range(len(my_sets[i])):
                    (my_graphs[i].plot(vertex_colors={'red':my_sets[i][j]}, figsize=3, axes=False) + text(str(n)+'_'+str(m+1)+'_'+str(my_names[i])+'_usual_energy_'+str(j),(0,-1.5))).save(str(n)+'_pictures/'+str(count)+'.png')
                    count += 1
            if i > 0:
                for j in range(len(my_sets[i])):
                    (my_graphs[i].plot(vertex_colors={'blue':my_sets[i][j]}, figsize=3, axes=False) + text(str(n)+'_'+str(m+1)+'_'+str(my_names[i])+'_usual_energy_'+str(j),(0,-1.5))).save(str(n)+'_pictures/'+str(count)+'.png')
                    count += 1
            for j in range(len(my_sets_R[i])):
                (my_graphs[i].plot(vertex_colors={'green':my_sets_R[i][j]}, figsize=3, axes=False) + text(str(n)+'_'+str(m+1)+'_'+str(my_names[i])+'_sum_energy_'+str(j),(0,-1.5))).save(str(n)+'_pictures/'+str(count)+'.png')
                count += 1
                
lengths = [5,6,7,8,9,10,11,12,16]
#lengths = [12]
#lengths = [5]
for l in lengths:
    WriteImages(l)



/var/folders/gl/1h79cnp95499q5rml5_h6r99yc60x5/T/ipykernel_5845/2097139968.py:210: DeprecationWarning: parameter 'sort' will be set to False by default in the future
See http://trac.sagemath.org/22349 for details.
  for v in G.vertices():
/var/folders/gl/1h79cnp95499q5rml5_h6r99yc60x5/T/ipykernel_5845/2097139968.py:211: DeprecationWarning: parameter 'sort' will be set to False by default in the future
See http://trac.sagemath.org/22349 for details.
  for u in G.vertices():


In [ ]:

#MultiSpectrum(10,4,1)

n = 8
#Path = graphs.PathGraph(n)
#PathSquared = Power(Path,2)
Cycle = graphs.CycleGraph(n)
#G = Power(Cycle,2)
G = WheelieGraph(n)





for m in range(n+1):
    if m > 1:
        M = DistinctGlobalMinimizers(G,m,One_over_R_potential)
        MN = DistinctGlobalMinimizers(G,m,Negative_R_potential)
        for i in range(len(M)):
            G.show(vertex_colors={'black':M[i]},figsize=3)
        for i in range(len(MN)):    
            G.show(vertex_colors={'green':MN[i]},figsize=3)

In [41]:
i=0
i+=1
i

1